In [1]:
import importlib

import pedect.config.BasicConfig
from pedect.config.BasicConfig import *

import pedect.dataset.CaltechDataset
from pedect.dataset.CaltechDataset import *

# import pedect.dataset.Dataset
# from pedect.dataset.Dataset import Dataset

import pedect.input.inputProcessing
from pedect.input.inputProcessing import *

import numpy as np

def reimport_all():
    importlib.reload(pedect.config.BasicConfig)
    importlib.reload(pedect.dataset.CaltechDataset)
    # importlib.reload(pedect.dataset.Dataset)
    importlib.reload(pedect.input.inputProcessing)

reimport_all()


In [2]:
config = BasicConfig()
print(config)

BasicConfig
Possible labels = {'people': (255, 0, 0), 'person-fa': (0, 0, 255), 'person': (0, 255, 0)}
Batchs size = 2
Steps per epoch = 100
Number of epochs = 10


In [3]:
chosenDataset = CaltechDataset()

In [5]:
video_path = ""
annotations_path = ""
videoSet = ""
videoNr = ""
if isinstance(chosenDataset, CaltechDataset):
    videoSet = "set00"
    videoNr = "V000"
    
video_path = chosenDataset.getVideoPath(videoSet, videoNr)
annotations_path = chosenDataset.getAnnotationsPath(videoSet, videoNr)
print("Video path = ", video_path)
print(annotations_path)
print("Annotations path = ", annotations_path, "hehe")

Video path = 

..\Data\caltech\set00\V000.seq

..\Data\caltech\annotations\set00\V000.vbb

In [8]:
annotations = read_vbb(annotations_path)
v = read_seq(video_path)

Format mjpeg detected only with low score of 25, misdetection possible!

In [12]:
print(getAllLabels(annotations))

{'people', 'person', 'person-fa'}

In [13]:
saveAnnotatedVideo(v, "videoWithAnnotation.mp4", config, annotations)

Format mjpeg detected only with low score of 25, misdetection possible!

deprecated pixel format used, make sure you did set range correctly

In [39]:
def make_generator(images, annotations, batchSize = config.batchSize):
    frames = [i for i in range(len(images))]
    out_images = []
    out_results = []
    while True:
        np.random.shuffle(frames)
        for frameNumber in frames:
            c_img = images[frameNumber]
            res = 1 if len(getAnnotationsForFrame(frameNumber, annotations)) > 0 else 0
            out_images.append(c_img)
            out_results.append(res)
            if len(out_images) >= batchSize:
                yield np.stack(out_images, 0) / 255.0, np.stack(out_results, 0)
                out_images, out_results = [], []

imageGenerator = make_generator([i for i in v][:100], annotations)
x, y = next(imageGenerator)

deprecated pixel format used, make sure you did set range correctly
 (repeated 1844 more times)

Format mjpeg detected only with low score of 25, misdetection possible!

deprecated pixel format used, make sure you did set range correctly

In [40]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Flatten

inputShape = x.shape[1:]
print(inputShape)

Using TensorFlow backend.


(480, 640, 3)

In [41]:
a = Input(shape = inputShape)
b = a
decrease = 5
if 480 % (2 ** decrease) != 0 or 640 % (2 ** decrease) != 0:
    assert(1 == 0)
for i in range(decrease):
    b = Conv2D(128, (3, 3), activation='relu', padding='same') (b)
    b = MaxPooling2D(pool_size = (2, 2))(b)
b = MaxPooling2D(pool_size=(480 / (2 ** decrease), 640 / (2 ** decrease)))(b)
b = Conv2D(1, (1, 1), activation='sigmoid') (b)
b = Flatten()(b)
model = Model(inputs=a, outputs=b)
model.summary()

_________________________________________________________________

Layer (type)                 Output Shape              Param #   

input_1 (InputLayer)         (None, 480, 640, 3)       0         

_________________________________________________________________

conv2d_1 (Conv2D)            (None, 480, 640, 128)     3584      

...

_________________________________________________________________

max_pooling2d_2 (MaxPooling2 (None, 120, 160, 128)     0         

_________________________________________________________________

conv2d_3 (Conv2D)            (None, 120, 160, 128)     147584    

_________________________________________________________________

In [42]:
import keras.backend as K
def tp_rate(y_true, y_pred, smooth = 1e-5):
    intersection = K.sum(K.flatten(y_true * y_pred))
    rly_true = K.sum(K.flatten(y_true))
    return (intersection + smooth) / (rly_true + smooth)
def fn_rate(y_true, y_pred, smooth = 1e-5):
    return tp_rate(1 - y_true, 1 - y_pred, smooth)

model.compile(Adam(), loss='binary_crossentropy', metrics = [tp_rate, fn_rate])

In [43]:
history = model.fit_generator(imageGenerator, verbose = 1, steps_per_epoch = config.stepsPerEpoch, epochs = config.nrEpochs)

Epoch 1/10

  1/100 [..............................]

 - ETA: 13:55 - loss: 0.6573 - tp_rate: 1.0000 - fn_rate: 0.5182



  2/100 [..............................]

...

 - ETA: 1:03 - loss: 0.7491 - tp_rate: 0.6394 - fn_rate: 0.6576



 37/100 [==========>...................]

 - ETA: 1:01 - loss: 0.7448 - tp_rate: 0.6491 - fn_rate: 0.6548

KeyboardInterrupt: 

In [79]:
rez, gt = [], []
for i in range(100):
    x, y = next(imageGenerator)
    rez.append(model.predict(x))
    gt.append(y)

In [80]:
a = [1 if i >= 0.5 else 0 for i in [i[0][0] for i in rez] + [i[1][0] for i in rez]]
b = [i[0] for i in gt] + [i[1] for i in gt]
x = [1 if a[i] == b[i] else 0 for i in range(len(a))]
print(x)
print(sum(x) / len(x))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.99
